In [84]:
import feedparser
from bs4 import BeautifulSoup as bs
import urllib
import urllib.request as req
import requests
from newspaper import Article
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma, Okt, Komoran
from konlpy.utils import pprint

In [85]:
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

feed = feedparser.parse('http://newssearch.naver.com/search.naver?where=rss&query=%EC%B6%98%EC%B2%9C&field=0&nx_search_query=&nx_and_query=&nx_sub_query=&nx_search_hlquery=&is_dts=0')

feed.keys()

dict_keys(['feed', 'entries', 'bozo', 'headers', 'href', 'status', 'encoding', 'bozo_exception', 'version', 'namespaces'])

## newspaper3k를 이용한 크롤링

link_list = []
for f in feed['entries']:
    if '춘천' in f.title or f.description:
        link_list.append(f.link)


link_list[0]

url = link_list[1]
article = Article(url, language='ko')
article.download()
article.parse()

print(article.title)
print('--')
print(article.text)

In [86]:
title_list = []
summ_list = []
doc_list = []

for f in feed['entries']:
    title_list.append(f.title)
    summ_list.append(f.summary)
    doc_list.append(f.title + f.summary)

In [87]:
for i in range(10):
    print(doc_list[i] + '\n'*2)

춘천시의회 후반기 원구성 마무리 '협치없는 민주당 독식'춘천시의회 후반기 원구성이 비판 여론에도 불구하고 전체 21석 중 13석을 차지한 더불어민주당 다수의 '협치없는 민주당 독식' 구도로 짜였다. 6일 춘천시의회는 오전 10시 시의회 본회의장에서 제302회 임시회를 열어 원구성을...


7일 더위 시작하는 소서(小暑), 대부분 30도 안팎…곳곳 소나기지역별 아침 최저기온은 △서울 21도 △인천 21도 △춘천 22도 △강릉 22도 △대전 22도 △대구 22도 △부산 21도 △전주 22도 △광주 22도 △제주 23도, 낮 최고기온은 △서울 28도 △인천 26도 △춘천 30도 △강릉 29도 △대전 28도...


[인사] 법무부 外정성수 △춘천보호관찰소장 최종철 △대전보호관찰소 천안지소장 배점호 △울산보호관찰소장 권기한... 이종근 △춘천국유림관리소장 김주미 ◆공정거래위원회 <전보> ◇과장급 △시장구조개선과장 박세민 ◆국립공원공단...


[인사] 보건복지부 / 법무부 / 국가과학기술인력개발원(KIRD) 등정성수 △춘천보호관찰소장 최종철 △대전보호관찰소 천안지소장 배점호 △울산보호관찰소장 권기한 △위치추적대전관제센터장 황남례 △법무부 치료처우과 김택준 △대구소년원 분류보호과장 김달곤 △광주소년원...


순풍가, 음식물 처리기 '다무스' 출시순풍가는 강릉시청, 거제시청, 춘천시 농업기술센터 등 전국 20여곳에서 공개시연회를 열었다. 주무부처인 환경부는 “순풍가 제품이 기존 처리방식의 단점을 획기적으로 개선했다”고 평가한 것으로 알려졌다. 순풍가...


춘천시의회 원 구성 마무리...민주당 의장단 6석 모두 차지강원 춘천시의회가 결국 민주당 의원들이 의장단 6석을 모두 차지하며 후반기 원구성을 마무리했습니다. 춘천시의회는 6일 본회의장에서 임시회를 열고, 통합당 몫이었던 부의장과 4석의 상임위원장 등 5명을 선출 완료했습니다....


산림조합-신한카드, 상반기 회원모집 프로모션 시상전국 1등은 전남 영광군산림조합이 차지했고 춘천시산림조합, 포항시산림조합 등이 

In [88]:
kkma = Kkma()
okt = Okt()
komoran = Komoran()

In [89]:
doc_kkma = []
doc_okt = []
doc_komoran = []

for i in range(len(doc_list)):
  kkma_nouns = ' '.join(kkma.nouns(doc_list[i]))
  okt_nouns = ' '.join(okt.nouns(doc_list[i]))
  komoran_nouns = ' '.join(komoran.nouns(doc_list[i]))

  doc_kkma.append(kkma_nouns)
  doc_okt.append(okt_nouns)
  doc_komoran.append(komoran_nouns)

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix_kkma = tfidf_vectorizer.fit_transform(doc_kkma)
tfidf_matrix_okt = tfidf_vectorizer.fit_transform(doc_okt)
tfidf_matrix_komoran = tfidf_vectorizer.fit_transform(doc_komoran)

doc_distance_kkma = (tfidf_matrix_kkma * tfidf_matrix_kkma.T)
doc_distance_okt = (tfidf_matrix_okt* tfidf_matrix_okt.T)
doc_distance_komoran = (tfidf_matrix_komoran * tfidf_matrix_komoran.T)

print('kkma 활용 유사도 분석을 위한 matrix의 dim : ' + str(doc_distance_kkma.get_shape()[0])+'*'+str(doc_distance_kkma.get_shape()[1]))
print(doc_distance_kkma.toarray(),'\n')

print('okt 활용 유사도 분석을 위한 matrix의 dim : ' + str(doc_distance_okt.get_shape()[0])+'*'+str(doc_distance_okt.get_shape()[1]))
print(doc_distance_okt.toarray(),'\n')

print('komoran 활용 유사도 분석을 위한 matrix의 dim : ' + str(doc_distance_komoran.get_shape()[0])+'*'+str(doc_distance_komoran.get_shape()[1]))
print(doc_distance_komoran.toarray())

kkma 활용 유사도 분석을 위한 matrix의 dim : 50*50
[[1.         0.02325689 0.0042288  ... 0.0052423  0.00510448 0.00395217]
 [0.02325689 1.         0.01281514 ... 0.04672246 0.04549416 0.04076572]
 [0.0042288  0.01281514 1.         ... 0.3818287  0.37179068 0.00185247]
 ...
 [0.0052423  0.04672246 0.3818287  ... 1.         0.97371068 0.00229644]
 [0.00510448 0.04549416 0.37179068 ... 0.97371068 1.         0.00223607]
 [0.00395217 0.04076572 0.00185247 ... 0.00229644 0.00223607 1.        ]] 

okt 활용 유사도 분석을 위한 matrix의 dim : 50*50
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.02555106 ... 0.0604562  0.05870829 0.05927062]
 [0.         0.02555106 1.         ... 0.45057231 0.43754539 0.        ]
 ...
 [0.         0.0604562  0.45057231 ... 1.         0.97108804 0.        ]
 [0.         0.05870829 0.43754539 ... 0.97108804 1.         0.        ]
 [0.         0.05927062 0.         ... 0.         0.         1.        ]] 

komoran 활용 유사도 분석을 위한 matrix의 d